## Normal Mode Analysis -- Huang-Rhys Factors and ReorgE

To read in atomic coordinates and freq calculation results for molecules, calc. Huang-Rhys factors and reorgE

In [222]:
import re
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import scipy.linalg as la
%matplotlib inline  
# maybe some 3D plot for the molecule?? Is it possible?
  

In [ ]:
### parse force const + modes: better factorize the code, + orientation code
class Nmode:   # just a container
    #tot_mode = N_mode #better (automatical) way doing this???
    
    def __init__(self,i_mode):
        self.i_mode = i_mode
        # parse neutral_freq.log
        self.kconst = 0.   # force const
        self.redM = 0.     # red. mass
        self.freq = 0.
        self.freqeV = 0.   #hbar*omega in eV
        #self.freqHz = 0.   # cm-1 --> Hz
        self.omega = 0.    # omega in xxx? (ev/hbar?)
        #self.MASS = np.zeros((N_atom*3,N_atom*3)) # for mass matrix, only diag. has values M^0.5, only need one copy
        self.CMODE = np.zeros(N_atom*3)
        self.norC = 0.   # normalization const for CMODE
        
        # parse ex.xyz
        #self.cordEX = np.zeros(N_atom*3)    # only need one copy
        # parse n.xyz
        #self.cordN = np.zeros(N_atom*3)     # only need one copy
        
        # after parsing above variables, these can be calc.
        self.deltaQ = 0.
        self.ReorgE = 0.
        self.HRfac = 0.    # Huang-Rhys factor = g^2
        self.gfac = 0.     # g factor = HRfac^0.5
        self.cfac = 0.     # Irene's coupling c in eV = g*hbar*omega
        
    #def xxx:
        
class GaussianParser:   # the methods which parse Gaussian and put values in the container
    
    def __init__(self, path):
        self.modes = []   # this will create a list of objects Nmode
        self.MASS = np.zeros((N_atom*3,N_atom*3))
        self.cordN = np.zeros(N_atom*3)
        self.cordEX = np.zeros(N_atom*3)
        self.dcord = np.zeros(N_atom*3)
        self.path = path
        
    def calc_reorgE(self):
        self.NormalModeCart()
        self.MassMatrix()
        self.NMCart2MWC()
        self.deltaRcart()
        self.deltaQ()
        self.NMA_ReorgE()
        
    def NormalModeCart(self): # store each mode i in self.modes[i].CMODE[]: each mode is a vector
        
        with open (self.path+n_freq_name) as file:  # n_freq_name: .log file
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'normal' in line:
                    line_s = num + 1  # record the start line
                    #print('find line_s: %d', line_s)
            if line_s is None:  # throw exception
                print('normal modes calculation not done!')  # correct???  
                #return 0   # funny!???
                exit()
            if N_mode%3 ==0:  # given as input now...
                nblock = int(N_mode/3)
                lastCol = 3
            else:
                nblock = int(N_mode/3) + 1
                lastCol = N_mode%3
            for i in range(nblock):
                for j in range(3):   # column numbers: 3 cols in a block
                    n_mode = i*3+j+1   # a counter for the N-th mode (for the object Nmode)
                    # store the values in Nmode object
                    self.modes.append(Nmode(n_mode))   # Nmode is an object defined above...
                    # freq in cm-1: write down the equation how gaussian get freq in cm-1!!
                    self.modes[n_mode-1].freq = float(lines[line_s+i*(N_atom+7)+2].split()[2+j])
                    self.modes[n_mode-1].freqeV = self.modes[n_mode-1].freq*0.000123981
                    self.modes[n_mode-1].kconst = float(lines[line_s+i*(N_atom+7)+4].split()[3+j]) 
                    self.modes[n_mode-1].redM = float(lines[line_s+i*(N_atom+7)+3].split()[3+j])
                    #print(self.modes[n_mode-1].kconst)
                    #convert cm-1 to Hz: 1 cm-1 = 2.998*10^10 Hz (check this!! this is light speed in cm/sec)
                    # cm-1 * light speed in cm/sec = freq in 1/sec
                    #self.modes[n_mode-1].freqHz = (self.modes[n_mode-1].freq*2.99793)*10**10  
                    #print(modes[n_mode-1].freq)   # for debug
                    # freqeV???
                    # omega???
                    for k in range(N_atom):
                        self.modes[n_mode-1].CMODE[3*k] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3]  # x
                        self.modes[n_mode-1].CMODE[3*k+1] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3+1] # y
                        self.modes[n_mode-1].CMODE[3*k+2] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3+2] # z
                        
    def MassMatrix(self):
        
        with open (self.path+n_freq_name) as file:  # n_freq_name: .log file
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'normal' in line:
                    line_s = num + 1  # record the start line
                    
            if line_s is None:  # throw exception
                print('normal modes calculation not done!')  # correct???  
                #return 0   # funny!???
                exit()
            if N_mode%3 ==0:  # given as input now...
                nblock = int(N_mode/3)
                lastCol = 3
            else:
                nblock = int(N_mode/3) + 1
                lastCol = N_mode%3
                
            for i in range(nblock):
                for k in range(N_atom):    
                    M_temp = lines[line_s+i*(N_atom+7)+7+k].split()[1]
                    #print(M_temp)
                    if M_temp == '1':  # 1 amu = 1.66*10^-27 kg
                        M_temp = 1.00797**0.5
                        #block_temp#M_temp = (1.00797*(1.66053904*10**-27))**0.5d
                    elif M_temp == '6':
                        M_temp = 12.011**0.5
                        #block_temp#M_temp = (12.011*(1.66053904*10**-27))**0.5
                    elif M_temp == '16':
                        M_temp = 32.06**0.5
                        #block_temp#M_temp = (32.06*(1.66053904*10**-27))**0.5
                    else:
                        print('atom Mass not exist!')  # correct??
                        exit()
                    self.MASS[3*k,3*k] = M_temp
                    self.MASS[3*k+1,3*k+1] = M_temp
                    self.MASS[3*k+2,3*k+2] = M_temp                       
                    #print(self.modes[n_mode-1].CMODE)    # for debug
        #for i in range(3*N_atom):
        #    print(self.MASS[i])
        
    def NMCart2MWC(self):
        for i in range(N_mode):
            for j in range(N_atom*3):
                self.modes[i].CMODE[j] = self.modes[i].CMODE[j]*self.MASS[j,j]
        
    def deltaRcart(self):
        with open (self.path+n_name) as file:
            lines = file.readlines()
            for i in range(N_atom):
                self.cordN[3*i] = round(float(lines[i+2].split()[1]),3)
                self.cordN[3*i+1] = round(float(lines[i+2].split()[2]),3)
                self.cordN[3*i+2] = round(float(lines[i+2].split()[3]),3)
            #print(self.cordN)
            
        with open (self.path+ex_name) as file:
            lines = file.readlines()
            for i in range(N_atom):
                self.cordEX[3*i] = round(float(lines[i+2].split()[1]),3)
                self.cordEX[3*i+1] = round(float(lines[i+2].split()[2]),3)
                self.cordEX[3*i+2] = round(float(lines[i+2].split()[3]),3)
            #print(self.cordEX)
        self.dcord = np.subtract(self.cordEX,self.cordN)
        #print('before:')
        #print("coordinate change: %s" % self.dcord)
        #block_temp#self.dcord *= 10**-10   # 1 Anstrom = 10^-10 m
        #print('after:')
        #print("dcord: %s" % self.dcord)
        
    def deltaQ(self):
        # normalize normal modes (again), then calc deltaQ
        for i in range(N_mode):
            # First, normalize NM_MWC (Mass-Weighted Coordinate)
            self.modes[i].norC = (1/np.dot(self.modes[i].CMODE,self.modes[i].CMODE))**0.5
            self.modes[i].CMODE *= self.modes[i].norC
            
            # deltaQ = C*M*deltaX
            # note: M*deltaX gives displacement vector in MWC
            # then C is already in MWC
            temp = np.dot(self.MASS,self.dcord)
            self.modes[i].deltaQ = np.dot(self.modes[i].CMODE, temp)
            
            #print("mode: %s ,deltaQ: %s" % (i+1, self.modes[i].deltaQ))
            
    def NMA_ReorgE(self):    
        #for debug, check tot ReorgE
        temp = 0.
        #count = 0
        for i in range(N_mode):
            ##if self.modes[i].freq > 0.:
            tem2 = self.modes[i].deltaQ
                # 1 J = 1 kg*m^2/s^2 = 6.24*10^18 eV
                #tem1: individual mode contributes to reorgE
                #tem1 = 0.5*(self.modes[i].freqHz**2)*(tem2**2)*(self.modes[i].norC**2)*(6.24150913*10**18)
                ##tem1 = 0.5*(self.modes[i].freqHz**2)*(tem2**2)*(6.24150913*10**18)
                #print(tem1)
                #temp: the sum of all modes to get reorgE
                #temp += 0.5*(self.modes[i].freqHz**2)*(tem2**2)*(self.modes[i].norC**2)
            tem1 = 0.5*(self.modes[i].kconst/self.modes[i].redM)*(tem2**2)*6.24150913 #*math.pi*2
            tem3 = tem1/self.modes[i].freqeV
            print("Mode %s ReorgE: %s, Hung-Rhys: %s" % (i+1,tem1,tem3))
            temp += 0.5*(self.modes[i].kconst/self.modes[i].redM)*(tem2**2)
                #temp += 0.5*(self.modes[i].freqHz)*(tem2**2)*(self.modes[i].norC**2)
                #count += 1
        ##temp = temp*(6.24150913*10**18)  #*(1.66053904*10**-27)*(10**-20)      #*((6.28318530718)**2)/0.529
        temp = temp*6.24150913 #*math.pi*2  #where is this 2pi factor from???
        print(temp)
        
                    
    #def print_all:
        
    #def plot_all:

In [82]:
#input file: Tetracene
N_atom = 30
N_mode = 3*N_atom -6
n_freq_name = 'tetra.log'
n_name = 'Tetracene_n.xyz'
ex_name = 'Tetracene_c.xyz'

path_ = './Gaussian_files/'
b3_Tetra = GaussianParser(path_)
b3_Tetra.calc_reorgE()

Mode 1 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 2 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 3 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 4 ReorgE: 2.33686545067e-08, Hung-Rhys: 1.15876295065e-06
Mode 5 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 6 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 7 ReorgE: 7.43023543723e-08, Hung-Rhys: 1.97404661178e-06
Mode 8 ReorgE: 0.00021945993567, Hung-Rhys: 0.00555804300318
Mode 9 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 10 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 11 ReorgE: 3.09683251996e-08, Hung-Rhys: 5.6054713708e-07
Mode 12 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 13 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 14 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 15 ReorgE: 2.26925843927e-08, Hung-Rhys: 3.62987599324e-07
Mode 16 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 17 ReorgE: 5.88867757626e-08, Hung-Rhys: 8.43676892392e-07
Mode 18 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 19 ReorgE: 1.00894497046e-07, Hung-Rhys: 1.31788109125e-06
Mode 20 ReorgE: 1.34141950434e-06, Hung-Rhys: 1.70684872914e-05
Mode 21 ReorgE: 1.07356010986e-06, Hung-Rhys: 1.34463010459e-05
Mode

In [83]:
#input file: Anthracene
N_atom = 24
N_mode = 3*N_atom -6
n_freq_name = 'ant.log'
n_name = 'ANT_n.xyz'
ex_name = 'ANT_c.xyz'

path_ = './Gaussian_files/'
b3_ANT = GaussianParser(path_)
b3_ANT.calc_reorgE()

Mode 1 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 2 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 3 ReorgE: 1.11633082418e-07, Hung-Rhys: 3.83070109007e-06
Mode 4 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 5 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 6 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 7 ReorgE: 4.11208440372e-07, Hung-Rhys: 8.41992504565e-06
Mode 8 ReorgE: 5.49955085863e-05, Hung-Rhys: 0.00111205965365
Mode 9 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 10 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 11 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 12 ReorgE: 2.13149479653e-07, Hung-Rhys: 3.2202527823e-06
Mode 13 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 14 ReorgE: 2.33928356051e-08, Hung-Rhys: 3.056955245e-07
Mode 15 ReorgE: 1.00205622607e-05, Hung-Rhys: 0.0001260557158
Mode 16 ReorgE: 3.29279076833e-08, Hung-Rhys: 4.01868676978e-07
Mode 17 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 18 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 19 ReorgE: 0.000229893169215, Hung-Rhys: 0.00242015879404
Mode 20 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 21 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 22 ReorgE: 4.7787103774e-07, Hu

In [84]:
#input file: pentacene: reorgE ~ 100 meV
N_atom = 36
N_mode = 3*N_atom -6
n_freq_name = 'pent.log'
n_name = 'pent_n.xyz'
ex_name = 'pent_c.xyz'

path_ = './Gaussian_files/'
b3_pent = GaussianParser(path_)
b3_pent.calc_reorgE()

Mode 1 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 2 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 3 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 4 ReorgE: 7.78762139167e-38, Hung-Rhys: 5.29429707458e-36
Mode 5 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 6 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 7 ReorgE: 3.11876648167e-37, Hung-Rhys: 1.05960832581e-35
Mode 8 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 9 ReorgE: 0.00088415515679, Hung-Rhys: 0.0269999331128
Mode 10 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 11 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 12 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 13 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 14 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 15 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 16 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 17 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 18 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 19 ReorgE: 3.34367554873e-37, Hung-Rhys: 5.4853504933e-36
Mode 20 ReorgE: 1.4439808693e-36, Hung-Rhys: 2.2798580421e-35
Mode 21 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 22 ReorgE: 0.0, Hung-Rhys: 0.0
Mode 23 ReorgE: 1.85217333111e-36, Hung-Rhys: 2.58213793686e-35
Mode 24 ReorgE: 4.2

In [76]:
#input file: Naphthalene: reorgE ~ 180 meV
N_atom = 18
N_mode = 3*N_atom -6
n_freq_name = 'naph.log'
n_name = 'naph_n.xyz'
ex_name = 'naph_c.xyz'

path_ = './Gaussian_files/'
b3_naph = GaussianParser(path_)
b3_naph.calc_reorgE()

Mode 1 ReorgE: 0.0
Mode 2 ReorgE: 0.0
Mode 3 ReorgE: 2.24923614067e-08
Mode 4 ReorgE: 0.0
Mode 5 ReorgE: 0.0
Mode 6 ReorgE: 0.0
Mode 7 ReorgE: 1.96484750593e-07
Mode 8 ReorgE: 0.00304235620194
Mode 9 ReorgE: 0.0
Mode 10 ReorgE: 1.43669861209e-06
Mode 11 ReorgE: 0.0
Mode 12 ReorgE: 0.000117692869081
Mode 13 ReorgE: 0.0
Mode 14 ReorgE: 0.0
Mode 15 ReorgE: 7.9947959033e-08
Mode 16 ReorgE: 0.0
Mode 17 ReorgE: 0.0
Mode 18 ReorgE: 2.67536496114e-06
Mode 19 ReorgE: 0.0
Mode 20 ReorgE: 0.0
Mode 21 ReorgE: 0.0
Mode 22 ReorgE: 0.0
Mode 23 ReorgE: 5.55839305218e-07
Mode 24 ReorgE: 0.000141489486477
Mode 25 ReorgE: 2.10819217406e-06
Mode 26 ReorgE: 2.00453139483e-07
Mode 27 ReorgE: 0.0
Mode 28 ReorgE: 0.00356188282996
Mode 29 ReorgE: 4.80162870927e-07
Mode 30 ReorgE: 2.80216673374e-08
Mode 31 ReorgE: 8.69362946497e-07
Mode 32 ReorgE: 6.79234237287e-07
Mode 33 ReorgE: 0.0299944568115
Mode 34 ReorgE: 8.31890710753e-07
Mode 35 ReorgE: 7.06001553391e-07
Mode 36 ReorgE: 0.00483071276868
Mode 37 ReorgE:

In [5]:
#input file: OT9
N_atom = 65
N_mode = 3*N_atom -6
n_freq_name = 'OT9_n_b3_planar.log'
n_name = 'OT9_n_b3.xyz'
ex_name = 'OT9_c_b3.xyz'

path_ = './'
b3_OT9 = GaussianParser(path_)
b3_OT9.calc_reorgE()

Mode 1 ReorgE: 1.28881140254e-09, Hung-Rhys: -4.75395383855e-07
Mode 2 ReorgE: 1.862579759e-09, Hung-Rhys: -6.9032080118e-07
Mode 3 ReorgE: 4.43955054267e-11, Hung-Rhys: -3.11411844076e-08
Mode 4 ReorgE: 2.38585083131e-10, Hung-Rhys: -1.73709220346e-07
Mode 5 ReorgE: 4.16319895861e-12, Hung-Rhys: -4.49600738684e-09
Mode 6 ReorgE: 3.7149959541e-11, Hung-Rhys: 7.38690367946e-08
Mode 7 ReorgE: 1.02204514916e-15, Hung-Rhys: 1.33514126565e-12
Mode 8 ReorgE: 2.59130786847e-11, Hung-Rhys: 2.90689234498e-08
Mode 9 ReorgE: 7.6050623981e-08, Hung-Rhys: 6.41463067854e-05
Mode 10 ReorgE: 7.54984747078e-10, Hung-Rhys: 6.20013006186e-07
Mode 11 ReorgE: 1.37560029301e-09, Hung-Rhys: 5.53028796167e-07
Mode 12 ReorgE: 5.61259787258e-38, Hung-Rhys: 1.81017020442e-35
Mode 13 ReorgE: 2.05539573424e-09, Hung-Rhys: 5.17422246914e-07
Mode 14 ReorgE: 1.1216238183e-06, Hung-Rhys: 0.000192335272277
Mode 15 ReorgE: 8.34003798405e-10, Hung-Rhys: 1.42092409697e-07
Mode 16 ReorgE: 9.95372535018e-11, Hung-Rhys: 1.20

## Energy Redistribution Analysis
input: Gaussian log file with frequencies (for normal coordinates), run-vel-1.xyz (velocity trajectory) 

In [223]:
# first step, principal axes...
# Read in coordinate file, determine and align molecule at principal axes...

class p_axes:
    def __init__(self,path):
        self.path = path
        self.cordN = np.zeros(N_atom*3)
        self.MASS = np.zeros(N_atom)
        self.Itensor = np.zeros((3,3))
        self.atoms = []
        
        ## for debug:
        self.rot_z = np.zeros((3,3))
        self.rot_y = np.zeros((3,3))
        
    def readC(self):
        with open (self.path+n_name) as file:
            lines = file.readlines()
            for i in range(N_atom):
                self.cordN[3*i] = round(float(lines[i+2].split()[1]),3)
                self.cordN[3*i+1] = round(float(lines[i+2].split()[2]),3)
                self.cordN[3*i+2] = round(float(lines[i+2].split()[3]),3)
                self.atoms.append(lines[i+2].split()[0])
                #print(M_temp)
                if self.atoms[i] == 'H':  # 1 amu = 1.66*10^-27 kg
                    self.MASS[i] = 1.00797
                elif self.atoms[i] == 'C':
                    self.MASS[i] = 12.011
                elif self.atoms[i] == 'S':
                    self.MASS[i] = 32.06
                else:
                    print('atom Mass not exist!')  # correct??                        
                    exit()                      
        #print(self.atoms)
        #print(self.MASS)
    
    # for debug: rotation along y and z axis...
    # theta: arc degree; ax: which axis
    def rot(self,theta,ax):
        #theta = math.pi/4
        if ax == 'z':
            ##rot_z = np.zeros((3,3))
            self.rot_z[0][0] = cos(theta)
            self.rot_z[0][1] = -sin(theta)
            self.rot_z[1][0] = sin(theta)
            self.rot_z[1][1] = cos(theta)
            self.rot_z[2][2] = 1
        elif ax == 'y':
            ##rot_y = np.zeros((3,3))
            self.rot_y[0][0] = cos(theta)
            self.rot_y[0][2] = sin(theta)
            self.rot_y[2][0] = -1*sin(theta)
            self.rot_y[2][2] = cos(theta)
            self.rot_y[1][1] = 1
        
        temp = np.zeros(3)
        for i in range(N_atom):
            temp1 = np.zeros(3)
            temp[0] = self.cordN[3*i]
            temp[1] = self.cordN[3*i+1]
            temp[2] = self.cordN[3*i+2]
            if ax == 'z':
                temp1 = np.dot(self.rot_z,temp)
            elif ax == 'y':
                temp1 = np.dot(self.rot_y,temp)
            self.cordN[3*i] = temp1[0]
            self.cordN[3*i+1] = temp1[1]
            self.cordN[3*i+2] = temp1[2]
        #print(self.rot_z)
    
    def I_tensor(self):
        Mxx = 0.
        Myy = 0.
        Mzz = 0.
        Mxy = 0.
        Mxz = 0.
        Myz = 0.
        for i in range(N_atom):
            Mxx += self.MASS[i] * (self.cordN[3*i+1]**2 + self.cordN[3*i+2]**2)
            Myy += self.MASS[i] * (self.cordN[3*i]**2 + self.cordN[3*i+2]**2)
            Mzz += self.MASS[i] * (self.cordN[3*i]**2 + self.cordN[3*i+1]**2)
            Mxy += -1 * self.MASS[i] * self.cordN[3*i] * self.cordN[3*i+1]
            Mxz += -1 * self.MASS[i] * self.cordN[3*i] * self.cordN[3*i+2]
            Myz += -1 * self.MASS[i] * self.cordN[3*i+1] * self.cordN[3*i+2]
        self.Itensor[0][0] = Mxx
        self.Itensor[0][1] = Mxy
        self.Itensor[0][2] = Mxz
        self.Itensor[1][0] = Mxy
        self.Itensor[1][1] = Myy
        self.Itensor[1][2] = Myz
        self.Itensor[2][0] = Mxz
        self.Itensor[2][1] = Myz
        self.Itensor[2][2] = Mzz
        
        eigvals, eigvecs = la.eig(self.Itensor)
        print(eigvals)
        print(eigvecs)
        
        ## Check short/long axes and directions
        l_axis = [self.cordN[2*3]-self.cordN[1*3], self.cordN[2*3+1]-self.cordN[1*3+1],self.cordN[2*3+2]-self.cordN[1*3+2]]
        norm = 0.
        for x in l_axis:
            norm += x**2
        l_axis = [(1/math.sqrt(norm))*x for x in l_axis ]
        s_axis = [self.cordN[5*3]-self.cordN[0*3], self.cordN[5*3+1]-self.cordN[0*3+1],self.cordN[5*3+2]-self.cordN[0*3+2]]
        for x in s_axis:
            norm += x**2
        s_axis = [(1/math.sqrt(norm))*x for x in s_axis ]
        z_axis = np.cross(l_axis,s_axis) 
        #print(l_axis)
        #print(s_axis)
        
        ## eig1, eig2, eig3 to determine which eigvec belong to which axis (short, long, z)
        eig1 = [eigvecs[0][0],eigvecs[1][0],eigvecs[2][0]]
        eig2 = [eigvecs[0][1],eigvecs[1][1],eigvecs[2][1]]
        eig3 = [eigvecs[0][2],eigvecs[1][2],eigvecs[2][2]]
        
        # first check short axis
        #ee1 = round((np.dot(s_axis,eig1)))
        #ee2 = round((np.dot(s_axis,eig2)))
        #ee3 = round((np.dot(s_axis,eig3)))
        rot_p = np.zeros((3,3))
                
        rot_p[0][0] = round((np.dot(l_axis,eig1)))*eig1[0] + round((np.dot(l_axis,eig2)))*eig2[0] + round((np.dot(l_axis,eig3)))*eig3[0]
        rot_p[0][1] = round((np.dot(l_axis,eig1)))*eig1[1] + round((np.dot(l_axis,eig2)))*eig2[1] + round((np.dot(l_axis,eig3)))*eig3[1]
        rot_p[0][2] = round((np.dot(l_axis,eig1)))*eig1[2] + round((np.dot(l_axis,eig2)))*eig2[2] + round((np.dot(l_axis,eig3)))*eig3[2]
        
        rot_p[1][0] = round((np.dot(s_axis,eig1)))*eig1[0] + round((np.dot(s_axis,eig2)))*eig2[0] + round((np.dot(s_axis,eig3)))*eig3[0]
        rot_p[1][1] = round((np.dot(s_axis,eig1)))*eig1[1] + round((np.dot(s_axis,eig2)))*eig2[1] + round((np.dot(s_axis,eig3)))*eig3[1]
        rot_p[1][2] = round((np.dot(s_axis,eig1)))*eig1[2] + round((np.dot(s_axis,eig2)))*eig2[2] + round((np.dot(s_axis,eig3)))*eig3[2]
        
        rot_p[2][0] = round((np.dot(z_axis,eig1)))*eig1[0] + round((np.dot(z_axis,eig2)))*eig2[0] + round((np.dot(z_axis,eig3)))*eig3[0]
        rot_p[2][1] = round((np.dot(z_axis,eig1)))*eig1[1] + round((np.dot(z_axis,eig2)))*eig2[1] + round((np.dot(z_axis,eig3)))*eig3[1]
        rot_p[2][2] = round((np.dot(z_axis,eig1)))*eig1[2] + round((np.dot(z_axis,eig2)))*eig2[2] + round((np.dot(z_axis,eig3)))*eig3[2]
        
        # rot cordN to align with short, long, z axes
        temp = np.zeros(3)
        for i in range(N_atom):
            temp1 = np.zeros(3)
            temp[0] = self.cordN[3*i]
            temp[1] = self.cordN[3*i+1]
            temp[2] = self.cordN[3*i+2]
            temp1 = np.dot(rot_p,temp)
            self.cordN[3*i] = temp1[0]
            self.cordN[3*i+1] = temp1[1]
            self.cordN[3*i+2] = temp1[2]
        
        ## for debug
        ##temp = np.zeros(3)
        ##temp1 = np.zeros(3)
        ##temp = np.dot(self.rot_y,self.rot_z)
        ##temp1 = np.dot(temp,rot_p)
        ##print(temp1)
        ## Testing end
        
    def print_xyz(self,out): # for debug...
        
        path_w = self.path + out + '.xyz'
        with open (path_w ,'w') as fout_:
            fout_.write( str(N_atom) )
            fout_.write('\n')
            fout_.write('\n')
            for i in range(N_atom):
                fout_.write(self.atoms[i] +' ' + str(self.cordN[3*i]) + ' ' + str(self.cordN[3*i+1]) + ' ' + str(self.cordN[3*i+2]) + '\n')        

class Nmode:   # just a container
    #tot_mode = N_mode #better (automatical) way doing this???
    
    def __init__(self,i_mode):
        self.i_mode = i_mode
        # parse neutral_freq.log
        self.kconst = 0.   # force const
        self.redM = 0.     # red. mass
        self.freq = 0.
        self.freqeV = 0.   #hbar*omega in eV
        #self.freqHz = 0.   # cm-1 --> Hz
        self.omega = 0.    # omega in xxx? (ev/hbar?)
        #self.MASS = np.zeros((N_atom*3,N_atom*3)) # for mass matrix, only diag. has values M^0.5, only need one copy
        self.CMODE = np.zeros(N_atom*3)
        self.norC = 0.   # normalization const for CMODE
        
        # parse ex.xyz
        #self.cordEX = np.zeros(N_atom*3)    # only need one copy
        # parse n.xyz
        #self.cordN = np.zeros(N_atom*3)     # only need one copy
        
        # after parsing above variables, these can be calc.
        self.deltaQ = 0.
        self.ReorgE = 0.
        self.HRfac = 0.    # Huang-Rhys factor = g^2
        self.gfac = 0.     # g factor = HRfac^0.5
        self.cfac = 0.     # Irene's coupling c in eV = g*hbar*omega
        
    #def xxx:
        
class GaussianParser:   # the methods which parse Gaussian and put values in the container
    
    def __init__(self, path):
        self.modes = []   # this will create a list of objects Nmode
        self.MASS = np.zeros((N_atom*3,N_atom*3))
        self.cordN = np.zeros(N_atom*3)
        self.cordEX = np.zeros(N_atom*3)
        self.dcord = np.zeros(N_atom*3)
        self.path = path                

    def NormalModeCart(self): # store each mode i in modes[i].CMODE[]: each mode is a vector
        
        with open (self.path+n_freq_name) as file:  # n_freq_name: .log file
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'normal' in line:
                    line_s = num + 1  # record the start line
                    #print('find line_s: %d', line_s)
            if line_s is None:  # throw exception
                print('normal modes calculation not done!')  # correct???  
                #return 0   # funny!???
                exit()
            if N_mode%3 ==0:  # given as input now...
                nblock = int(N_mode/3)
                lastCol = 3
            else:
                nblock = int(N_mode/3) + 1
                lastCol = N_mode%3
            for i in range(nblock):
                for j in range(3):   # column numbers: 3 cols in a block
                    n_mode = i*3+j+1   # a counter for the N-th mode (for the object Nmode)
                    # store the values in Nmode object
                    self.modes.append(Nmode(n_mode))   # Nmode is an object defined above...
                    # freq in cm-1: write down the equation how gaussian get freq in cm-1!!
                    self.modes[n_mode-1].freq = float(lines[line_s+i*(N_atom+7)+2].split()[2+j])
                    self.modes[n_mode-1].freqeV = self.modes[n_mode-1].freq*0.000123981
                    self.modes[n_mode-1].kconst = float(lines[line_s+i*(N_atom+7)+4].split()[3+j]) 
                    self.modes[n_mode-1].redM = float(lines[line_s+i*(N_atom+7)+3].split()[3+j])
                    #print(self.modes[n_mode-1].kconst)
                    #convert cm-1 to Hz: 1 cm-1 = 2.998*10^10 Hz (check this!! this is light speed in cm/sec)
                    # cm-1 * light speed in cm/sec = freq in 1/sec
                    #self.modes[n_mode-1].freqHz = (self.modes[n_mode-1].freq*2.99793)*10**10  
                    #print(modes[n_mode-1].freq)   # for debug
                    # freqeV???
                    # omega???
                    for k in range(N_atom):
                        self.modes[n_mode-1].CMODE[3*k] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3]  # x
                        self.modes[n_mode-1].CMODE[3*k+1] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3+1] # y
                        self.modes[n_mode-1].CMODE[3*k+2] = lines[line_s+i*(N_atom+7)+7+k].split()[2+j*3+2] # z
                        
    def MassMatrix(self):
        
        with open (self.path+n_freq_name) as file:  # n_freq_name: .log file
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'normal' in line:
                    line_s = num + 1  # record the start line
                    
            if line_s is None:  # throw exception
                print('normal modes calculation not done!')  # correct???  
                #return 0   # funny!???
                exit()
            if N_mode%3 ==0:  # given as input now...
                nblock = int(N_mode/3)
                lastCol = 3
            else:
                nblock = int(N_mode/3) + 1
                lastCol = N_mode%3
                
            for i in range(nblock):
                for k in range(N_atom):    
                    M_temp = lines[line_s+i*(N_atom+7)+7+k].split()[1]
                    #print(M_temp)
                    if M_temp == '1':  # 1 amu = 1.66*10^-27 kg
                        M_temp = 1.00797**0.5
                        #block_temp#M_temp = (1.00797*(1.66053904*10**-27))**0.5d
                    elif M_temp == '6':
                        M_temp = 12.011**0.5
                        #block_temp#M_temp = (12.011*(1.66053904*10**-27))**0.5
                    elif M_temp == '16':
                        M_temp = 32.06**0.5
                        #block_temp#M_temp = (32.06*(1.66053904*10**-27))**0.5
                    else:
                        print('atom Mass not exist!')  # correct??
                        exit()
                    self.MASS[3*k,3*k] = M_temp
                    self.MASS[3*k+1,3*k+1] = M_temp
                    self.MASS[3*k+2,3*k+2] = M_temp                       
                    #print(self.modes[n_mode-1].CMODE)    # for debug
        #for i in range(3*N_atom):
        #    print(self.MASS[i])
        
    def NMCart2MWC(self):
        for i in range(N_mode):
            for j in range(N_atom*3):
                self.modes[i].CMODE[j] = self.modes[i].CMODE[j]*self.MASS[j,j]        
        
        


In [226]:
#input file: OT9
N_atom = 65
n_name = 'OT9_n_b3.xyz'

path_ = './'
b3_OT9 = p_axes(path_)
b3_OT9.readC()
b3_OT9.rot(math.pi/4,'z')
b3_OT9.rot(math.pi/4,'y')
#b3_OT9.I_tensor()
b3_OT9.print_xyz('test-z45y45')


In [221]:
#input file: OT9
N_atom = 65
n_name = 'OT9_1MD.xyz'

path_ = './MD_files/'
MD_1OT9 = p_axes(path_)
MD_1OT9.readC()
MD_1OT9.I_tensor()
MD_1OT9.print_xyz('test-o')


[  36225.26795921+0.j   77272.85823711+0.j  111726.66752557+0.j]
[[-0.99962216  0.02710675  0.0045561 ]
 [ 0.0052065   0.02397182  0.99969908]
 [-0.02698937 -0.99934507  0.02410389]]
[[-0.99962216  0.0052065  -0.02698937]
 [ 0.0045561   0.99969908  0.02410389]
 [ 0.02710675  0.02397182 -0.99934507]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


## Molecular Builder

In [7]:
class Molecule:
    
    def __init__(self,Name,path_):
        self.Name = Name 
        self.path_ = path_
        
        #try parse path_ to get rid of file name
        #ugly, need polishment!
        self.directory = ''
        for x in path_.split('/')[1:-1]:
            self.directory = self.directory + '/' + x
        self.directory = self.directory + '/'
        
        with open (path_) as file_:
            self.coord_ = []
            self.atoms_ = []
            #self.Natom_ = file_.readlines()[0]            
            lines_ = file_.readlines()
            self.Natom_ = lines_[0]
            lines_ = lines_[2:int(self.Natom_)+2] #int(self.Natom_)+2 should be more general than -1 or something
            for line in lines_:
                split_ = [ float(x) for x in line.split()[1:] ]
                self.coord_.append( split_ )
                self.atoms_.append( line.split()[0] )
        #print (self.atoms_)
        self.center = []
        self.coord_c = []
        self.combined_Natom_ = self.Natom_
        self.combined_atoms_ = self.atoms_
        self.combined_coord_ = []
           
    def print_coord(self):
        print (self.coord_)
        
    def calc_center(self):  # calc_center for whole molecule
        if self.coord_c:
            coord_t = np.array(self.coord_c).transpose()
        else:
            coord_t = np.array(self.coord_).transpose()
        # print (coord_t)
        x_ = np.mean( [ float(x) for x in coord_t[0] ] )
        y_ = np.mean( [ float(x) for x in coord_t[1] ] )
        z_ = np.mean( [ float(x) for x in coord_t[2] ] )
        self.center = [x_, y_, z_]
        print (self.center)
        
            # python does not support function overloading. 
            # given a atom_list [i,j,k,...], return the center for (atom i, atom j, atom k, and ...)
    def calc_center_atom(self, atom_list):      
        coord_a =[]
        for i in atom_list:
            if self.coord_c:
                coord_a.append( self.coord_c[i-1] )
            else:
                coord_a.append( self.coord_[i-1] )
        coord_a_t =np.array(coord_a).transpose()
        x_ = np.mean( [ float(x) for x in coord_a_t[0] ] )
        y_ = np.mean( [ float(x) for x in coord_a_t[1] ] )
        z_ = np.mean( [ float(x) for x in coord_a_t[2] ] )
        print (x_, y_, z_)        
    
    def orien_center(self):  # move center to (0.0 0.0 0.0)
        #first check if already centered
        if not self.center:
            self.calc_center()
        
        # try polish here!!
        k = 0
        while (self.center[k]<10**-6):
            self.center[k] = 0.0
            k = k+1
            if (k>2):
                break

        for i in range(len(self.coord_)):
            self.coord_c.append( [x1-x2 for (x1,x2) in zip(self.coord_[i],self.center)] )
        #print( self.coord_c)
        #try deal with the degits of self.coord_c

    def image_xz(self):
        #first check if centered (in the future, maybe also check if align long-short axis)
        if not self.coord_c:
            self.orien_center()
        #print (len(self.coord_))
        for i in range( len(self.coord_) ):
            #print (self.coord_c[i][1])
            self.coord_c[i][1] = -1*self.coord_c[i][1]
        #print( self.coord_c)

    def save_xyz(self, option):
        if not self.coord_c:
            #self.orien_center()
            self.coord_c = self.coord_
        if option == 'imaged':
            self.image_xz()
        if option == 'combined':
            Natom_ = str(self.combined_Natom_) + '\n'
            atoms_ = self.combined_atoms_
        else:
            Natom_ = self.Natom_
            atoms_ = self.atoms_
        coord_ = {
            'default': self.coord_ ,
            'centered': self.coord_c ,
            'shifted': self.coord_c ,
            'imaged': self.coord_c,
            'combined': self.combined_coord_ ,   # add in the future
        }[option]
        
        #print (coord_)
        
        #write files
        path_w = self.directory + self.Name + '-' + option + '.xyz'
        with open (path_w ,'w') as fout_:
            fout_.write( Natom_ )
            fout_.write('\n')
            for i in range(len(atoms_)):
                fout_.write(atoms_[i] +' ' + str(coord_[i][0]) + ' ' + str(coord_[i][1]) + ' ' + str(coord_[i][2]) + '\n')        
    
    def shift_z(self, dist):
        if not self.coord_c:
            #self.orien_center()
            self.coord_c = self.coord_
        for i in range( len(self.coord_) ):
            self.coord_c[i][2] = self.coord_c[i][2] + dist
        #print( self.coord_c)        
        
    def combine_molecule(self, obj):
        #print ( obj.Name )
        self.combined_Natom_ = int(self.combined_Natom_) + int(obj.Natom_)
        print (self.combined_Natom_)
        if not self.center:
            self.calc_center()
        if not obj.center:
            obj.calc_center()
        self.combined_atoms_ = self.combined_atoms_ + obj.atoms_
        if not self.combined_coord_:
            self.combined_coord_ = self.coord_c + obj.coord_c
        else:
            self.combined_coord_ = self.combined_coord_ + obj.coord_c
        #print(self.combined_atoms_)
        #print('length:',len(self.combined_atoms_))
        #print(self.combined_coord_)
        
    #def orien_x(self):  # from center to an assigned point as x-axis 
        
    #def orien_y(self):  # rotate along x-axis, this seems tricky!
        
    #def rot_aboutZ(self, degree):
        
    #def rot_aboutXYZ(self, axis, degree):
    
    #def calc_avg_plane(self):
    
    #def calc_long_short_axis(self):
        #first check if already calc_avg_plane

In [355]:
path_C60_OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60_OT9-shifted.xyz'
C60_OT9 = Molecule('C60_OT9', path_C60_OT9)
#C60_center = [1, 2, 4, 6, 8, 38, 31, 32, 34, 36]

unit = [1, 2, 3, 4, 6]
OT9_center1 = []
OT9_center2 = []
for i in range(9):
    num1 = 60+ (i*65) + 28
    num2 = 60+ (i*65) + 57
    unit1 = []
    unit2 = []
    for j in unit:
        unit1.append(num1+j)
        unit2.append(num2+j)
    print(unit1)
    print(unit2)
    OT9_center1.append(unit1)
    OT9_center2.append(unit2)

for k in range(len(OT9_center1)):
    C60_OT9.calc_center_atom(OT9_center1[k])
    C60_OT9.calc_center_atom(OT9_center2[k])


#C60_OT9.calc_center_atom(C60_center)
#C60_OT9.shift_z(7.0)
#C60_OT9.calc_center_atom(C60_center)
# C60_OT9.calc_center()
# print (C60_OT9.Natom_)
#C60_OT9.save_xyz('shifted')


[89, 90, 91, 92, 94]
[118, 119, 120, 121, 123]
[154, 155, 156, 157, 159]
[183, 184, 185, 186, 188]
[219, 220, 221, 222, 224]
[248, 249, 250, 251, 253]
[284, 285, 286, 287, 289]
[313, 314, 315, 316, 318]
[349, 350, 351, 352, 354]
[378, 379, 380, 381, 383]
[414, 415, 416, 417, 419]
[443, 444, 445, 446, 448]
[479, 480, 481, 482, 484]
[508, 509, 510, 511, 513]
[544, 545, 546, 547, 549]
[573, 574, 575, 576, 578]
[609, 610, 611, 612, 614]
[638, 639, 640, 641, 643]
15.691537 -0.348744892309 6.99983736923
-15.691537 -0.348744692309 7.00043516923
15.691537 0.348744892309 10.7998373692
-15.691537 0.348744692309 10.8004351692
15.691537 -0.348744892309 14.5998373692
-15.691537 -0.348744692309 14.6004351692
15.691537 0.348744892309 18.3998373692
-15.691537 0.348744692309 18.4004351692
15.691537 -0.348744892309 22.1998373692
-15.691537 -0.348744692309 22.2004351692
15.691537 0.348744892309 25.9998373692
-15.691537 0.348744692309 26.0004351692
15.691537 -0.348744892309 29.7998373692
-15.691537 -0.348

In [304]:
OT9_11.shift_z(11.4)
OT9_.combine_molecule(OT9_11)
OT9_21.image_xz()
OT9_21.shift_z(15.2)
OT9_.combine_molecule(OT9_21)
OT9_12.shift_z(19.0)
OT9_.combine_molecule(OT9_12)
OT9_22.image_xz()
OT9_22.shift_z(22.8)
OT9_.combine_molecule(OT9_22)
OT9_13.shift_z(26.6)
OT9_.combine_molecule(OT9_13)
OT9_23.image_xz()
OT9_23.shift_z(30.4)
OT9_.combine_molecule(OT9_23)
OT9_.save_xyz('combined')

[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
320
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
385
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
450
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
515
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
580
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
645


In [303]:
path_OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/OT9_b3_n.xyz'
OT9_ = Molecule('OT9', path_OT9)
OT9_1 = Molecule('OT9', path_OT9)
OT9_2 = Molecule('OT9', path_OT9)
OT9_11 = Molecule('OT9', path_OT9)
OT9_21 = Molecule('OT9', path_OT9)
OT9_12 = Molecule('OT9', path_OT9)
OT9_22 = Molecule('OT9', path_OT9)
OT9_13 = Molecule('OT9', path_OT9)
OT9_23 = Molecule('OT9', path_OT9)
#OT9_.print_coord()
#print (OT9_.Natom_)
OT9_.orien_center()
OT9_.image_xz()
#print (OT9_.directory)
#OT9_.save_xyz('centered')
#OT9_.save_xyz('imaged')
OT9_1.shift_z(3.8)
OT9_.combine_molecule(OT9_1)
OT9_.combine_molecule(C60_)
OT9_2.image_xz()
OT9_2.shift_z(7.6)
OT9_.combine_molecule(OT9_2)

#for i in range(3):  #why this doesn't work???
#OT9_1.shift_z(7.6)
#OT9_.combine_molecule(OT9_1)
#OT9_2.shift_z(7.6)
#OT9_.combine_molecule(OT9_2)

#OT9_.save_xyz('combined')

[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
130
190
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
255


In [317]:
path_ = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-neutral-B3opt.xyz'
#xyz = []  # xyz should be defined before put into constructor!
C60_ = Molecule('C60', path_)
C60_.calc_center()
#C60_.print_coord()
#C60_.orien_center()
#C60_.print_coord()
#C60_.orien_center()
#print (C60_.Natom_)
#C60_.shift_z(-7.)

[-7.4014868308343768e-18, 3.333333335279557e-08, 5.0000000031043729e-08]
